In [1]:
import os, subprocess
import json
import uproot
import awkward as ak
import numpy as np
from coffea import processor, util, hist
import pandas as pd
import pickle
import matplotlib.pyplot as plt

/opt/conda/lib/python3.8/site-packages/coffea/util.py:154: FutureWarning: In coffea version v0.8.0 (target date: 31 Dec 2022), this will be an error.
(Set coffea.deprecations_as_errors = True to get a stack trace now.)
ImportError: coffea.hist is deprecated
  warnings.warn(message, FutureWarning)


In [2]:
with open('lumi.json') as f:
  lumis = json.load(f)

with open('xsec.json') as f:
  xs = json.load(f)

with open('pmap.json') as f:
  pmap = json.load(f)

ddbthr = 0.64

In [3]:
year='2016APV'
picklename = year+'/templates.pkl'
if not os.path.isfile(picklename):
    print("You need to create the pickle "+picklename)
else:
    templates = pickle.load(open(picklename,'rb')).sum('msd1','pt1').integrate('systematic','nominal').integrate('mjj',overflow='allnan')

In [4]:
templates.identifiers('process')

[<StringBin (EWKW) instance at 0x7f215a34f160>,
 <StringBin (EWKZ) instance at 0x7f215a34f040>,
 <StringBin (QCD) instance at 0x7f21d861bd60>,
 <StringBin (QCDHT) instance at 0x7f21d861b700>,
 <StringBin (VBF) instance at 0x7f215a34f250>,
 <StringBin (VBFDipoleRecoilOff) instance at 0x7f215a34f1f0>,
 <StringBin (VV) instance at 0x7f21d85cb760>,
 <StringBin (VVNLO) instance at 0x7f215a34f2b0>,
 <StringBin (WH) instance at 0x7f215a34f2e0>,
 <StringBin (Wjets) instance at 0x7f21d861b670>,
 <StringBin (ZH) instance at 0x7f215a34f370>,
 <StringBin (Zjets) instance at 0x7f21d861b9a0>,
 <StringBin (ZjetsHT) instance at 0x7f21d861b910>,
 <StringBin (data) instance at 0x7f215a34f430>,
 <StringBin (ggF) instance at 0x7f215a34f1c0>,
 <StringBin (muondata) instance at 0x7f215a34f490>,
 <StringBin (singlet) instance at 0x7f21d861b7f0>,
 <StringBin (ttH) instance at 0x7f215a34f3d0>,
 <StringBin (ttbar) instance at 0x7f21d861bb80>]

In [5]:
mc = ['QCD','Wjets','Zjets','Zjetsbb','EWKW','EWKZ','ttbar','singlet','VV','ggF','VBF','WH','ZH','ttH']
cutflow = pd.DataFrame(columns=['ggffail','ggfpass','vbffail','vbfpass'],index=mc)

In [6]:
templates.integrate('process','QCD').integrate('region',['signal-ggf','signal-vbf']).sum('genflavor','ddb1').values()

{(): 6500182.762043123}

In [7]:
for p in mc:
    print(p)
    if "Zjets" in p:
        continue
    
    cutflow['ggffail'][p] = templates.sum('genflavor').integrate('ddb1',int_range=slice(0,ddbthr)).integrate('region','signal-ggf').integrate('process',p).values()[()]
    cutflow['ggfpass'][p] = templates.sum('genflavor').integrate('ddb1',int_range=slice(ddbthr,1)).integrate('region','signal-ggf').integrate('process',p).values()[()]
    cutflow['vbffail'][p] = templates.sum('genflavor').integrate('ddb1',int_range=slice(0,ddbthr)).integrate('region','signal-vbf').integrate('process',p).values()[()]
    cutflow['vbfpass'][p] = templates.sum('genflavor').integrate('ddb1',int_range=slice(ddbthr,1)).integrate('region','signal-vbf').integrate('process',p).values()[()]
    
cutflow['ggffail']['Zjets'] = templates.integrate('genflavor',int_range=slice(0,3)).integrate('ddb1',int_range=slice(0,ddbthr)).integrate('region','signal-ggf').integrate('process','Zjets').values()[()]
cutflow['ggfpass']['Zjets'] = templates.integrate('genflavor',int_range=slice(0,3)).integrate('ddb1',int_range=slice(ddbthr,1)).integrate('region','signal-ggf').integrate('process','Zjets').values()[()]
cutflow['vbffail']['Zjets'] = templates.integrate('genflavor',int_range=slice(0,3)).integrate('ddb1',int_range=slice(0,ddbthr)).integrate('region','signal-vbf').integrate('process','Zjets').values()[()]
cutflow['vbfpass']['Zjets'] = templates.integrate('genflavor',int_range=slice(0,3)).integrate('ddb1',int_range=slice(ddbthr,1)).integrate('region','signal-vbf').integrate('process','Zjets').values()[()]

cutflow['ggffail']['Zjetsbb'] = templates.integrate('genflavor',int_range=slice(3,4)).integrate('ddb1',int_range=slice(0,ddbthr)).integrate('region','signal-ggf').integrate('process','Zjets').values()[()]
cutflow['ggfpass']['Zjetsbb'] = templates.integrate('genflavor',int_range=slice(3,4)).integrate('ddb1',int_range=slice(ddbthr,1)).integrate('region','signal-ggf').integrate('process','Zjets').values()[()]
cutflow['vbffail']['Zjetsbb'] = templates.integrate('genflavor',int_range=slice(3,4)).integrate('ddb1',int_range=slice(0,ddbthr)).integrate('region','signal-vbf').integrate('process','Zjets').values()[()]
cutflow['vbfpass']['Zjetsbb'] = templates.integrate('genflavor',int_range=slice(3,4)).integrate('ddb1',int_range=slice(ddbthr,1)).integrate('region','signal-vbf').integrate('process','Zjets').values()[()]



QCD
Wjets
Zjets
Zjetsbb
EWKW
EWKZ
ttbar
singlet
VV
ggF
VBF
WH
ZH
ttH


In [8]:
cutflow.astype('int')

,ggffail,ggfpass,vbffail,vbfpass
QCD,6188044,6691,304841,605
Wjets,101840,185,2419,3
Zjets,35821,98,879,3
Zjetsbb,5109,823,103,21
EWKW,3975,23,570,5
EWKZ,780,23,99,5
ttbar,18703,93,552,2
singlet,2577,16,108,0
VV,1072,16,29,0
ggF,71,16,3,0


In [9]:
(cutflow['ggffail']['ggF']+cutflow['ggfpass']['ggF']+cutflow['ggffail']['VBF']+cutflow['ggfpass']['VBF'])/sum(cutflow['ggffail']+cutflow['ggfpass'])

1.729318449024163e-05

In [10]:
(cutflow['vbffail']['ggF']+cutflow['vbfpass']['ggF']+cutflow['vbffail']['VBF']+cutflow['vbfpass']['VBF'])/sum(cutflow['vbffail']+cutflow['vbfpass'])

6.0257033371232667e-05

In [11]:
cutflow.astype('int').to_latex(buf=year+'/cutflow-final.tex')

/tmp/ipykernel_64/3630590848.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  cutflow.astype('int').to_latex(buf=year+'/cutflow-final.tex')
